In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [8]:
print(torch.__version__)

1.8.1+cu111


### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

In [20]:
batch = numpy.array([val_dataset[0],val_dataset[1]])

In [32]:
inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
inp = torch.Tensor(inp)

In [60]:
scene = batch[0]
scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]

array([[[3235.39526367, 1929.49365234],
        [3235.85620117, 1929.80456543],
        [3235.94506836, 1929.90905762],
        [3237.28198242, 1931.51367188],
        [3237.79418945, 1931.74328613],
        [3238.26123047, 1932.32592773],
        [3238.71533203, 1932.63842773],
        [3239.05664062, 1932.74816895],
        [3239.63549805, 1933.54748535],
        [3240.15014648, 1934.04931641],
        [3240.55395508, 1934.35949707],
        [3241.04003906, 1934.9621582 ],
        [3241.54272461, 1935.44311523],
        [3242.02246094, 1935.65124512],
        [3242.36499023, 1935.9407959 ],
        [3242.83886719, 1936.31408691],
        [3243.12109375, 1936.5065918 ],
        [3243.38110352, 1936.68774414],
        [3243.88574219, 1937.12597656]]])

In [66]:
batch[0]['track_id']

array([[['00000000-0000-0000-0000-000000000000'],
        ['00000000-0000-0000-0000-000000000000'],
        ['00000000-0000-0000-0000-000000000000'],
        ...,
        ['00000000-0000-0000-0000-000000000000'],
        ['00000000-0000-0000-0000-000000000000'],
        ['00000000-0000-0000-0000-000000000000']],

       [['00000000-0000-0000-0000-000000000007'],
        ['00000000-0000-0000-0000-000000000007'],
        ['00000000-0000-0000-0000-000000000007'],
        ...,
        ['00000000-0000-0000-0000-000000000007'],
        ['00000000-0000-0000-0000-000000000007'],
        ['00000000-0000-0000-0000-000000000007']],

       [['00000000-0000-0000-0000-000000000009'],
        ['00000000-0000-0000-0000-000000000009'],
        ['00000000-0000-0000-0000-000000000009'],
        ...,
        ['00000000-0000-0000-0000-000000000009'],
        ['00000000-0000-0000-0000-000000000009'],
        ['00000000-0000-0000-0000-000000000009']],

       ...,

       [['dummy51'],
        ['dummy51'],


In [49]:
p = numpy.array(numpy.zeros((seq_len,batch_size)))
p

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [61]:
batch_size = 2
seq_len = 19
feat_len = 2
p = numpy.array(numpy.zeros((seq_len,batch_size, feat_len)))
for i,scene in enumerate(batch):
    for j in range(seq_len):
        p[j,i,:] = scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],j]

### Create a loader to enable batch processing

In [63]:
batch_sz = 4

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ seq_len x batch_sz x feature] """
    batch_size = batch_sz
    inp_seq_len = batch[0]['p_in'][0].shape[0]
    out_seq_len = batch[0]['p_out'][0].shape[0]
    feat_len = 2
    inp = numpy.array(numpy.zeros((inp_seq_len,batch_size, feat_len)))
    out
    for i,scene in enumerate(batch):
        for j in range(inp_seq_len):
            inp[j,i,:] = scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],j]
        for j  in range(out_seq_len):
            out[j,i,:] = scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],j]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# class Transformer(nn.Module): 
#     def __init__(self):
        
#         super(Transformer, self).__init__()
        
#         self.transformer = nn.Transformer(d_model=2,
#                                          batch_first = True)

#     def forward(self, x, hidden):
#         embedded = F.relu(self.linear(x))
#         hidden = self.lstm(embedded, hidden)
#         return hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(model, device, train_loader, optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #output whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        model.train()
        optimizer.zero_grad()
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        loss = 0
        
        out = model(_input, target)

        
        loss += torch.sqrt(criterion(out[:, :2], target[:, di, :2]))

        # Backpropagate
        loss.backward()
        optimizer.step()
        
#         file1 = open("loss_steps.txt", "a")  # append mode
#         file1.write(str(loss.item()) + ",")
#         file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))
    

In [11]:
device = "cuda"

transformer = nn.Transformer(d_model=2, nhead=2)
# encoder = EncoderRNN(input_size=2)
# decoder = DecoderRNN(output_size=2)

# encoder = nn.DataParallel(encoder)
# decoder = nn.DataParallel(decoder)

# encoder.to(device)
# decoder.to(device)

# encoder_optimizer = torch.optim.Adam(encoder.parameters())
# decoder_optimizer = torch.optim.Adam(decoder.parameters())

optimizer = torch.optim.Adam(transformer.parameters())

num_epoch = 1

for epoch in range(1, num_epoch + 1):
    train(transformer, device, train_agent_loader, optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

RuntimeError: the batch number of src and tgt must be equal

In [ ]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [ ]:
output = infer_absolute(test_loader, encoder, decoder)

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

In [ ]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [ ]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

In [ ]:
df2.to_csv("outputs6epb4all.csv", index=True)